In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("rmisra/news-category-dataset")

print("Path to dataset files:", path)

In [ ]:
# Dataset was uploaded and loaded successfully in earlier steps.
# Proceeding to preprocessing and analysis.


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
# Step 1: Import libraries
import pandas as pd
import json

# Step 2: Load the dataset
with open('News_Category_Dataset_v3.json', 'r') as f:
    data = [json.loads(line) for line in f]

# Step 3: Convert to DataFrame
df = pd.DataFrame(data)

# Step 4: Sample for Colab (max 2000 rows)
df_sample = df.sample(n=2000, random_state=42)

# Step 5: Clean dataset
text_column = 'short_description'  # or 'headline' if preferred
category_column = 'category'

df_clean = df_sample.dropna(subset=[text_column, category_column])
df_clean = df_clean.rename(columns={text_column: 'content', category_column: 'category'})

# Step 6: Check categories and sample
print(df_clean['category'].value_counts())

# Step 7: Save prepared dataset
df_clean.to_csv('newsbot_dataset.csv', index=False)
print("✅ Dataset ready as 'newsbot_dataset.csv'")


In [ ]:
import pandas as pd
import json

# Load the JSON file
with open('News_Category_Dataset_v3.json', 'r') as f:
    data = [json.loads(line) for line in f]

# Convert to DataFrame
df = pd.DataFrame(data)

# Quick look at the data
print(df.shape)
print(df.columns)
print(df.head())


In [ ]:
# Sample the dataset to ~2000 articles
df_sample = df.sample(n=2000, random_state=42)

# Select the text and category columns
text_column = 'short_description'  # or 'headline' if you prefer
category_column = 'category'

# Remove rows with missing values in text or category
df_clean = df_sample.dropna(subset=[text_column, category_column])

# Rename columns for consistency
df_clean = df_clean.rename(columns={text_column: 'content', category_column: 'category'})

# Quick check
print(f"Dataset shape after sampling and cleaning: {df_clean.shape}")
print("Category distribution:")
print(df_clean['category'].value_counts())


In [ ]:
# Install and import libraries
!pip install -U spacy
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import spacy

# Load English model
nlp = spacy.load("en_core_web_sm")
stop_words = set(stopwords.words('english'))

# Preprocessing function
def preprocess(text):
    # 1. Lowercase
    text = text.lower()
    # 2. Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # 3. Tokenize
    tokens = word_tokenize(text)
    # 4. Remove stopwords
    tokens = [t for t in tokens if t not in stop_words]
    # 5. Lemmatize
    doc = nlp(" ".join(tokens))
    tokens = [token.lemma_ for token in doc]
    return " ".join(tokens)

# Apply preprocessing to your dataset
df_clean['processed_content'] = df_clean['content'].apply(preprocess)

# Check the first few rows
df_clean.head()


In [ ]:
import nltk
nltk.download('punkt_tab')


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create TF-IDF vectorizer
tfidf = TfidfVectorizer(max_features=2000)  # Limit features to 2000 for Colab efficiency

# Fit and transform the processed text
X = tfidf.fit_transform(df_clean['processed_content'])

# Check the shape
print(f"TF-IDF matrix shape: {X.shape}")

# Labels
y = df_clean['category']
print(f"Number of categories: {len(y.unique())}")
print("Categories:", y.unique())


In [ ]:
from sklearn.model_selection import train_test_split

# 80% training, 20% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training samples: {X_train.shape[0]}, Testing samples: {X_test.shape[0]}")


In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score

# Initialize and train model
model = MultinomialNB()
model.fit(X_train, y_train)

# Predict on test set
y_pred = model.predict(X_test)

# Evaluate
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


In [ ]:
!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Initialize analyzer
analyzer = SentimentIntensityAnalyzer()

# Function to get sentiment scores
def get_sentiment(text):
    scores = analyzer.polarity_scores(text)
    # Compound score is the overall sentiment
    if scores['compound'] >= 0.05:
        return 'positive'
    elif scores['compound'] <= -0.05:
        return 'negative'
    else:
        return 'neutral'

# Apply sentiment analysis
df_clean['sentiment'] = df_clean['content'].apply(get_sentiment)

# Check results
df_clean[['content', 'category', 'sentiment']].head(10)


In [ ]:
import spacy

# Load English model (should already be installed)
nlp = spacy.load("en_core_web_sm")

# Function to extract named entities
def extract_entities(text):
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

# Apply NER to your dataset
df_clean['entities'] = df_clean['content'].apply(extract_entities)

# Check the first few rows
df_clean[['content', 'entities']].head(5)


In [ ]:
from collections import Counter

all_entities = [ent for sublist in df_clean['entities'] for ent in sublist]
entity_types = [ent[1] for ent in all_entities]

entity_count = Counter(entity_types)
print(entity_count)


In [ ]:
def newsbot_pipeline(article):
    # 1. Preprocess
    processed = preprocess(article)

    # 2. TF-IDF transform
    vector = tfidf.transform([processed])

    # 3. Predict category
    category = model.predict(vector)[0]

    # 4. Sentiment analysis
    sentiment = get_sentiment(article)

    # 5. Named entity recognition
    entities = extract_entities(article)

    # 6. Return all insights
    return {
        'category': category,
        'sentiment': sentiment,
        'entities': entities
    }

# Test with a new article
sample_article = "Apple announced a new iPhone today, and the tech world is buzzing with excitement."
result = newsbot_pipeline(sample_article)
print(result)


In [ ]:
import matplotlib.pyplot as plt

# Count articles per category
category_counts = df_clean['category'].value_counts()

# Plot
plt.figure(figsize=(8,5))
category_counts.plot(kind='bar', color='skyblue')
plt.title("Number of Articles per Category")
plt.xlabel("Category")
plt.ylabel("Count")
plt.show()


In [ ]:
sentiment_counts = df_clean['sentiment'].value_counts()

plt.figure(figsize=(6,4))
sentiment_counts.plot(kind='pie', autopct='%1.1f%%', colors=['lightgreen','lightcoral','lightgray'])
plt.title("Sentiment Distribution")
plt.ylabel("")
plt.show()


In [ ]:
from collections import Counter

# Flatten list of all entities
all_entities = [ent[0] for sublist in df_clean['entities'] for ent in sublist]
entity_counts = Counter(all_entities)

# Top 10 entities
top_entities = entity_counts.most_common(10)

# Plot
labels, values = zip(*top_entities)
plt.figure(figsize=(8,5))
plt.bar(labels, values, color='orange')
plt.xticks(rotation=45)
plt.title("Top 10 Named Entities Across Articles")
plt.show()


# 3. Text Preprocessing
We clean and normalize the text by removing punctuation, converting to lowercase,
removing stopwords, and lemmatizing words.


In [ ]:
# See all column names in your dataset
print(df.columns)


In [ ]:

def fast_preprocess(text):
    tokens = text.lower().split()  # simple split
    return [t for t in tokens if t.isalpha() and t not in stop_words]

df['processed_content'] = (df['headline'] + ' ' + df['short_description']).apply(fast_preprocess)


In [ ]:
# Combine processed tokens back into string
df['processed_text_str'] = df['processed_content'].apply(lambda x: ' '.join(x))

# Quick check
df[['processed_text_str']].head()


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize vectorizer
tfidf = TfidfVectorizer(max_features=1000, ngram_range=(1,2), stop_words='english')

# Fit and transform
X_tfidf = tfidf.fit_transform(df['processed_text_str'])

# Features
features = tfidf.get_feature_names_out()
print("Top 20 features:", features[:20])


In [ ]:
import pandas as pd

# Convert TF-IDF to DataFrame
tfidf_df = pd.DataFrame(X_tfidf.toarray(), columns=features)
tfidf_df['category'] = df['category']

# Top 5 terms per category
category_top_terms = {}
for cat in df['category'].unique():
    avg_tfidf = tfidf_df[tfidf_df['category']==cat].drop('category', axis=1).mean()
    top_terms = avg_tfidf.sort_values(ascending=False).head(5).index.tolist()
    category_top_terms[cat] = top_terms

print("Top 5 terms per category:")
for cat, terms in category_top_terms.items():
    print(f"{cat}: {terms}")


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score

# Encode labels if needed
y = df['category']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Train classifier
clf = MultinomialNB()
clf.fit(X_train, y_train)

# Predictions
y_pred = clf.predict(X_test)

# Evaluate
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

# Apply NER to first 5 articles
for doc in df['processed_text_str'].head(5):
    spacy_doc = nlp(doc)
    entities = [(ent.text, ent.label_) for ent in spacy_doc.ents]
    print("Entities:", entities)


In [ ]:
from textblob import TextBlob

# Simple sentiment polarity
df['sentiment'] = df['processed_text_str'].apply(lambda x: TextBlob(x).sentiment.polarity)

# Quick check
print(df[['processed_text_str','sentiment']].head())


In [ ]:
df.to_csv('newsbot_dataset_final.csv', index=False)
print("✅ Dataset saved as 'newsbot_dataset_final.csv'")


In [ ]:
import matplotlib.pyplot as plt

df['category'].value_counts().plot(kind='bar', figsize=(8,5), color='skyblue')
plt.title('Number of Articles per Category')
plt.xlabel('Category')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.show()


In [ ]:
import seaborn as sns

# Example: top 5 terms for each category (already computed in Step 3)
for cat, terms in category_top_terms.items():
    print(f"{cat}: {terms}")


In [ ]:
top_terms = tfidf_df[tfidf_df['category']=='Politics'].drop('category', axis=1).mean().sort_values(ascending=False).head(10)
sns.barplot(x=top_terms.values, y=top_terms.index, palette='viridis')
plt.title('Top 10 TF-IDF Terms - Politics')
plt.show()


In [ ]:
df['sentiment'].hist(bins=20, figsize=(8,5), color='salmon')
plt.title('Sentiment Polarity Distribution')
plt.xlabel('Polarity')
plt.ylabel('Frequency')
plt.show()


Insights from NewsBot Analysis

After processing and analyzing the dataset, several patterns emerge:

Category Distribution – The dataset contains a balanced mix of news categories, with [Category X] having the most articles and [Category Y] the least. This ensures our model can learn effectively across different topics.

Top TF-IDF Terms – Each category has distinct key terms that characterize its content. For example, “government,” “policy,” and “election” dominate political articles, while “technology,” “innovation,” and “software” are frequent in tech news. These terms highlight the model’s ability to capture category-specific vocabulary.

Sentiment Analysis – Overall sentiment skews [positive/neutral/negative], with business and entertainment news showing more positive sentiment and politics showing mixed or neutral tones. This demonstrates the system’s ability to extract meaningful emotional context from articles.

Practical Value – By combining preprocessing, TF-IDF, sentiment, and entity recognition, NewsBot can quickly categorize articles, identify key entities, and highlight trends, making it useful for media monitoring, market intelligence, and research applications.

In [ ]:
df.to_csv('newsbot_dataset_final.csv', index=False)
print("✅ Dataset saved as 'newsbot_dataset_final.csv'")


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

# Use the cleaned and processed text column
X = df_clean['headline']  # change to 'short_description' if you prefer
y = df_clean['category']

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert text to TF-IDF features
vectorizer = TfidfVectorizer(max_features=2000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Train Naive Bayes classifier
model = MultinomialNB()
model.fit(X_train_tfidf, y_train)

# Predict on test set
y_pred = model.predict(X_test_tfidf)

# Print classification results
print(classification_report(y_test, y_pred))
